# Dash

구조
1. Layout
 * Dash HTML Components - https://dash.plotly.com/dash-html-components
 * Dash Core Components - https://dash.plotly.com/dash-core-components
2. Callback
 * Input : 입력값으로 사용할 부분을 식별하는 구성요소. Layout에서 작성한 id를 인자로 입력해주면 해당 영역의 값이 입력값으로 사용된다.
 * Output : 출력값으로 반환할 부분을 식별하는 구성요소. Layout에서 작성한 id를 인자로 입력해주면, 해당 영역으로 출력값이 반환된다.

In [1]:
# !pip install dash

In [2]:
# !pip install dash_auth
# 권한 부여 
'''
USERNAME_PASSWORD = {'user':'pw1234'}
app = dash.Dash()
auth = dash_auth.BasicAuth(app, USERNAME_PASSWORD)
server = app.server
app.layout = html.Div()
'''

"\nUSERNAME_PASSWORD = {'user':'pw1234'}\napp = dash.Dash()\nauth = dash_auth.BasicAuth(app, USERNAME_PASSWORD)\nserver = app.server\napp.layout = html.Div()\n"

## library install

In [3]:
import pandas as pd
import numpy as np

# Dash packages
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

from plotly.colors import DEFAULT_PLOTLY_COLORS   # chart default colors

<ipython-input-3-d1a354005ebf>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-d1a354005ebf>:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### 대시보드 1

In [4]:
# 데이터 호출
df = pd.read_csv('./data/Sales data/Data.csv')
df.head()

,OrderID,OrderDate,Gender,AgeGroup,Channel,ShipDate,ItemCode,MapCode,Quantity,Revenue,Cost,Category,Item Type,Region,Country,Code2,Code3,Latitude,Longitude
0,100013196,2020-03-20,Female,30s,Online,2020-03-22,D10002,34,46,35576.6,29647.155,Office,Office Supplies,Asia,Taiwan,TW,TWN,23.697810,120.960515
1,100061261,2018-06-20,Female,50s,Offline,NaN,D10001,5,69,10222.8,8518.990,Home,Household,Asia,China,CN,CHN,35.861660,104.195397
2,100071990,2020-06-20,Female,20s,Online,2020-06-22,D10001,3,53,31731.5,26442.930,Home,Household,Europe,Belgium,BE,BEL,50.503887,4.469936
3,100074136,2019-04-12,Male,40s,Online,2019-04-15,C10003,31,84,30739.6,19720.180,Foods,Vegetables,Asia,South Korea,KR,KOR,35.907757,127.766922
4,100103318,2018-03-27,Male,30s,Offline,NaN,C10004,7,56,62652.9,19689.517,Foods,Fruits,America,Cuba,CU,CUB,21.300000,-80.000000


In [5]:
# 이익(Margin) 생성
df['Margin'] = df['Revenue'] - df['Cost']

In [6]:
# 연도, 월 변수 생성
df['year'] = df['OrderDate'].str.slice(start = 0, stop = 4)
df['month'] = df['OrderDate'].str.slice(start = 5, stop = 7)
# 데이터 정렬
df = df.sort_values(by = ['Region','Channel','Category','Item Type','year','month','Gender'])

In [7]:
years = list(df['year'].unique())
years.sort()

### App & Layout

In [ ]:
# App structure
app = dash.Dash(__name__)
app.title = ("Dashboard | Sales Data")
server = app.server

# App layout
app.layout = html.Div([
    
    # Main Title
    html.H2('Sales Dashboard with Dash', style={'textAlign': 'center', 'marginBottom':10, 'marginTop':10}),
    
    # 영역 나누기 - Left
    html.Div([
        
        ### Pie by Channel, Gender, AgeGroup
        html.Div(className='Pie',
                 children=[
                     html.Div(dcc.Graph(id='channel'), style={'float':'left', 'display':'inline-block', 'width':'33%'}),
                     html.Div(dcc.Graph(id='gender'), style={'float':'left', 'display':'inline-block', 'width':'33%'}),
                     html.Div(dcc.Graph(id='agegroup'), style={'float':'right', 'width':'33%'})
                 ]),
        
        ### Indicater by Region, Bar by Country
        html.Div(className='Indicator & Bar',
                 children=[
                     html.Div(dcc.Graph(id='idc_africa'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_america'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_asia'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_europe'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_oceania'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='country'), style={'float':'right', 'width':'40%'})
                 ]),
                
        ### Line by YM, Radar by Category
        html.Div(className='Line',
                 children=[
                     html.Div(dcc.Graph(id='line'), style={'float':'left', 'display':'inline-block', 'width':'60%'}),
                     html.Div(dcc.Graph(id='radar'), style={'float':'right', 'width':'40%'})
                 ])  
        ], style={'float':'left', 'display':'inline-block', 'width':'65%'}),
    
    # 영역 나누기 - Right
    html.Div([
        html.Div(children=[
                    html.Div(dcc.Dropdown(id = 'id_year',
                                         options=[{'label':i, 'value':i} for i in years],
                                         value = max(years),
                                         style={'width':'50%'})),
                    html.Div(dcc.Graph(id='map')),
                    html.Div(dcc.Graph(id='sankey'))
            ])
    ], style={'float':'right', 'width':'35%'})
])

In [ ]:
# App structure
app = dash.Dash(__name__)                # 앱 정의
app.title = ('Dashboard | Sales Data')   # 웹 브라우저 탭
server = app.server                      # 서버 정의

# App layout
app.layout = html.Div([                  # Layout 정의
    
    # Main Title
    html.H2('Sales Dashboard with Dash', style={'textAlign': 'center', 'marginBottom':10, 'marginTop':10}),
    
    # 영역 나누기 - Left
    html.Div([
        
        ### Pie by Channel, Gender, AgeGroup
        html.Div(className='Pie',
                 children=[
                     html.Div(dcc.Graph(id='channel'), style={'float':'left', 'display':'inline-block', 'width':'33%'}),
                     html.Div(dcc.Graph(id='gender'), style={'float':'left', 'display':'inline-block', 'width':'33%'}),
                     html.Div(dcc.Graph(id='agegroup'), style={'float':'right', 'width':'33%'})
                 ]),
        
        ### Indicater by Region, Bar by Country
        html.Div(className='Indicator & Bar',
                 children=[
                     html.Div(dcc.Graph(id='idc_africa'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_america'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_asia'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_europe'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='idc_oceania'), style={'float':'left', 'display':'inline-block', 'width':'12%'}),
                     html.Div(dcc.Graph(id='country'), style={'float':'right', 'width':'40%'})
                 ]),
                
        ### Line by YM, Radar by Category
        html.Div(className='Line',
                 children=[
                     html.Div(dcc.Graph(id='line'), style={'float':'left', 'display':'inline-block', 'width':'60%'}),
                     html.Div(dcc.Graph(id='radar'), style={'float':'right', 'width':'40%'})
                 ])  
        ], style={'float':'left', 'display':'inline-block', 'width':'65%'}),
    
    # 영역 나누기 - Right
    html.Div([
        html.Div(children=[
                    html.Div(dcc.Dropdown(id = 'id_year',
                                         options=[{'label':i, 'value':i} for i in years],
                                         value = max(years),
                                         style={'width':'50%'})),
                    html.Div(dcc.Graph(id='map')),
                    html.Div(dcc.Graph(id='sankey'))
            ])
    ], style={'float':'right', 'width':'35%'})
])

cols = DEFAULT_PLOTLY_COLORS

########## Pie's 
@app.callback([Output('channel',  'figure'), 
               Output('gender', 'figure'), 
               Output('agegroup',    'figure')], 
              [Input('id_year', 'value')])

def update_output(val):
        
    # loop value's
    pies = ['Channel', 'Gender', 'AgeGroup']
    
    # data by channel, gender, agegroup
    figures = []
    
    for i in range(len(pies)):
        df_fig = df[df['year'] == val]
        df_fig = df_fig.loc[:,[pies[i],'Revenue']].groupby(by = [pies[i]], as_index = False).sum()
        
        # hover text
        df_fig['text'] = round(df_fig['Revenue']/1000000,1).astype(str) + 'M'
        
        
        trace = go.Pie(labels = df_fig[pies[i]],
                       values = df_fig['Revenue'],
                       name = '',
                       text = df_fig['text'],
                       textinfo = 'label+percent',
                       hovertemplate = "[%{label}]<br> Revenue: %{text}<br> Rate: %{percent}",
                       hoverinfo='text',
                       insidetextorientation = 'tangential',   # textinfo 타입 (tangential / auto / horizontal / radial)
                       hole = 0.4, 
                       marker_colors = cols  # pie color
                       )
        data = [trace]
        
        layout = go.Layout(title=pies[i], title_x=0.5, title_xanchor='center', showlegend=False, 
                       height=250, margin=dict(l=50, r=50, b=10, t=50)
                      )
        
        figure = go.Figure(data, layout)
        figures.append(figure)

    return figures[0], figures[1], figures[2]

# indicator
########## by Region
@app.callback([Output('idc_africa',  'figure'), 
               Output('idc_america', 'figure'), 
               Output('idc_asia',    'figure'), 
               Output('idc_europe',  'figure'), 
               Output('idc_oceania', 'figure')], 
              [Input('id_year', 'value')])

def update_output(val):
        
    # reg - unique value's
    reg = df['Region'].unique()
    
    # data by Region
    figures = []
    
    for i in range(len(reg)):
        df_fig = df[(df['year'] == val) & (df['Region'] == reg[i])]
        df_fig = round(df_fig.loc[:,['Revenue','Margin']].sum(),1)
        
        values = df_fig['Revenue']
        deltas = df_fig['Margin']
        
        trace = go.Indicator(mode = 'number+delta',
                             value = values,
                             number = dict(font_size = 35),   # font size fixed (안하면 반응형으로 크기 제각각)
                             delta = dict(reference = values - deltas,
                                          font_size = 20,
                                          relative = False,
                                          increasing_color = '#3078b4', increasing_symbol = '',
                                          decreasing_color = '#d13b40', decreasing_symbol = '',
                                          position = 'top'),
                             title = dict(text = reg[i], font_size = 20)
                            )
        data = [trace]
        
        layout = go.Layout(height=310)
        figure = go.Figure(data, layout)
        figures.append(figure)

    return figures[0], figures[1], figures[2], figures[3], figures[4]

# Run App
if __name__=='__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Nov/2021 00:59:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 00:59:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 01:01:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Nov/2021 01:01:23] "POST /_dash-update-component HTTP/1.1" 